<a href="https://colab.research.google.com/github/hyunicecream/Natural-Language-Processing-NLP-/blob/main/KoGPT2(gen_text).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!nvidia-smi

Wed Aug 11 02:41:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!pip install --upgrade mxnet>=1.6.0
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

In [9]:
# -*- coding: utf-8 -*-
"""8-12.KoGPT2(gen_text).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ABZelGJI7VxTiwsnmD6Fy3o85hxXfmw0
"""

# GPT2의 TFGPT2LMHeadModel 모델을 이용한 언어 생성
# TFGPT2LMHeadModel : The GPT2 Model transformer with a language modeling head on top 
#                     (linear layer with weights tied to the input embeddings).
# !pip install --upgrade mxnet>=1.6.0
# !pip install gluonnlp
# !pip install transformers
# !pip install sentencepiece
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer, SentencepieceDetokenizer
from transformers import TFGPT2LMHeadModel
import tensorflow as tf

In [15]:
MY_PATH = '/content/drive/MyDrive/머신러닝/GPT/'
MODEL_PATH = MY_PATH + 'gpt_ckpt'
TOKENIZER_PATH = MY_PATH + 'gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
detokenizer = SentencepieceDetokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token = None,
                                               sep_token = None,
                                               cls_token = None,
                                               unknown_token = '<unk>',
                                               padding_token = '<pad>',
                                               bos_token = '<s>',
                                               eos_token = '</s>')
# vocab --> Vocab(size=50000, unk="<unk>", reserved="['<pad>', '<s>', '</s>']")

In [16]:
# tokenizer 연습
# 참고 : https://nlp.gluon.ai/api/modules/data.html
toked = tokenizer('안녕 하세요')   # tokenizer가 잘못 만들어진 듯 ... 한글자씩 ?
print(toked)

toked_idx = vocab(toked)
print(toked_idx)

toked = vocab.to_tokens(toked_idx)
print(toked)

detoked = detokenizer(toked)
print(detoked)

''.join(toked).replace('▁', ' ')

['▁', '안', '녕', '▁', '하', '세', '요']
[47437, 47541, 48439, 47437, 47451, 47530, 47580]
['▁', '안', '녕', '▁', '하', '세', '요']
안녕 하세요


' 안녕 하세요'

In [17]:
model = TFGPT2LMHeadModel.from_pretrained(MODEL_PATH)
model.summary()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/머신러닝/GPT/gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgp_t2lm_head_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFGPT2MainLayer multiple                  124242432 
Total params: 124,242,432
Trainable params: 124,242,432
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 모델의 seed 입력 문장 생성
tok = tokenizer('이때')   # tok = ['▁', '이', '때']
tok_idx = [vocab[vocab.bos_token]] + vocab[tok]     # tok_idx = [0, 47437, 47438, 47675]
input_ids = tf.convert_to_tensor(tok_idx)[None, :]  # 텐서로 변환

In [19]:
# 모델의 출력
output = model.generate(input_ids, max_length=50)

output

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


<tf.Tensor: shape=(1, 50), dtype=int32, numpy=
array([[    0, 47437, 47438, 47675,   211,  3918,   164,   831,   638,
          148,   222, 47929, 47454,   164,   831,   638,   222, 47929,
        47454,   164,   831,   638,   222, 47929, 47454,   164,   831,
          638,   222, 47929, 47454,   164,   831,   638,   222, 47929,
        47454,   164,   831,   638,   222, 47929, 47454,   164,   831,
          638,   222, 47929, 47454,   164]], dtype=int32)>

In [20]:
# 모델의 출력을 문자열로 변환
out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

이때부터 나는 내 모든 것을 다 바쳐서 내 모든 것을 바쳐서 내 모든 것을 바쳐서 내 모든 것을 바쳐서 내 모든 것을 바쳐서 내 모든 것을 바쳐서 내 모든 것을 바쳐서 내


In [21]:
# Beam search
output = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


이때부터  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지  ⁇ 그제까지


In [22]:
# 연속된 단어가 나오는 것을 방지함. no_repeat_ngram_size = 2
output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


이때쯤이면  ⁇ 그제 같을 것 같다. "어머나, 이 녀석!!!!!!!" ^^* Subject 2말괄량이프린세스 9장 *063* 로맨스


In [23]:
# top_k sampling
output = model.generate(input_ids, max_length=50, do_sample = True, top_k=100, temperature=0.8)

out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


이때야말로 그런 생각이 간절했던 것도 같다. 또 다른 전문가들은 정부가 고용시장 변화를 살피지 않고 무작정 밀어붙이기만을 할 경우 '고용 없는 성장'을 가져온다는 점을 강조할 것으로 보인다. 전문가들은 "미국의 경기 회복


In [24]:
# top_p sampling
output = model.generate(input_ids, max_length=50, do_sample = True, top_p=0.9, temperature=0.8)

out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


이때부터 사귄 건 아니고 내가 원하는 걸 다 받아 줄 테니깐 그때는 내가 원하는 게 뭐든 다 받아 줄께 자? 미안해 자? 잘 자 자?


In [25]:
# top_k & top_p sampling
output = model.generate(input_ids, max_length=50, do_sample = True, top_k=100, top_p=0.9, temperature=0.8)

out_tok_idx = output.numpy().tolist()[0]   # output token 인덱스
out_tok = vocab.to_tokens(out_tok_idx)     # token 인덱스를 token 문자로 변환
out_text = detokenizer(out_tok)            # 출력 문자열로 decode
print(out_text)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


이때부터 미류는 미류의 마음을 알아채고 있었다. "미류, 왜 그래? 이제 그만 자...  ⁇ , 그 시간에 잠자구 있잖아.  ⁇ , 그 시간에
